원문: Moler, Cleve B. [Numerical computing with MATLAB](https://www.mathworks.com/moler/chapters.html). Society for Industrial and Applied Mathematics, 2008.

## 5.5 QR 분해

만일 모든 매개변수가 선형적으로 나타나고 관측값의 수가 기저함수의 수 보다 많다면, 우리는 최소자승 문제를 가지고 있는 것이 된다. 설계행렬 $X$ 는 $m \times n$ 이고 $m>n$ 이다. 우리가 풀고 싶은 문제는 다음과 같다.

$$X \beta \approx y$$

그러나 이 연립방정식은 과결정이다. 방정식이 미지수보다 많다. 따라서 우리는 이 연립방정식의 엄밀하게 풀 수 없다. 사실, 우리는 최소자승 관점에서 푼다:
$$ min_\beta ||X \beta - y ||$$
과결정된 연립방정식을 푸는 한가지 가능한 이론적 접근 방법으로 양변을 $X^T$ 로 곱하는 방법이 있다. 이렇게 하면 해당 연립방정식이 $n \times n$ 의 *정규 방정식* 이라고 알려진 형태로 줄어든다. 

$$X^T X \beta = X^T y$$

관찰값의 수는 수천이지만 미지수의 수는 몇개 밖에 안될 경우, 설계행렬 $X$는 상당히 커지지만, 해당 행렬 $X^TX$는 작아진다. $y$ 는 $X$의 각 열로 생성된 공간에 투사된다. 이러한 이론적 접근 을 이어가서, 만일 기저함수가 독립이라면, $X^T X$는 비특이 행렬이 되고 매개변수는 다음과 같이 구해진다.

$$\beta = (X^T X)^{-1} X^T y $$

선형 최소 자승 문제를 푸는 이 공식은 거의 모든 통계와 수치 해석 교과서에 나타난다. 그러나, 몇가지 바람직하지 않은 면이 이 이론적 접근 방법에 있다. 우리는 이미 역행렬을 사용해서 연립방정식을 푸는 것이 가우스 소거법으로 푸는 것 보다 더 수고로우면서 덜 정확하다는 것을 보았다. 그러나, 더 중요한 것은, 정규 방정식은 그 원래 과결정 연립방정식 보다 특이행렬에 더 가깝다는 것이다. 사실, 행렬의 조건수가 제곱이 된다:

$$\kappa (X^T X) = \kappa(X)^2$$

유한 정밀도 연산에서는, 실제로 $X$의 해당 열이 독립적이 된다고 하더라도 정규 방정식의 $(X^TX)$은 특이행렬이 되어 $(X^TX)^{-1}$ 이 존재하지 않게 될 수 있다.

한 극단적인 예로 다음과 같은 설계 행렬을 고려해 보라.

\begin{equation}
X=
  \begin{bmatrix}
    1 & 1 \\
    \delta & 0 \\
    0 & \delta
  \end{bmatrix}
\end{equation}

만일 $\delta$ 가 작다면, $X$의 두 열은 거의 평행하겠지만 선형 독립일 것이다. 해당 정규 방정식에서는 상황이 더 나쁘게 된다:

\begin{equation}
X^T X =
  \begin{bmatrix}
    1+ \delta^2 & 1 \\
    1 & 1 +\delta^2
  \end{bmatrix}
\end{equation}

만일 $|\delta| < 10^{-8}$ 인 경우, 행렬 $(X^TX)$를 배정도 부동소숫점으로 연산하면 정확히 특이행렬이 되고 고전적인 교과서 공식 안에서 요구되는 역행렬은 존재하지 않는다.

QR 분해라고 알려져 있는 *직교화* 알고리듬을 주로 사용하여 정규 방정식을 피하는 방법이 있다. 


In [ ]:
import numpy as np
import numpy.linalg as na


delta = 1e-8
X = np.matrix([[1, 1],
              [delta, 0],
              [0, delta]])

y = np.matrix([[0, 1, 2]]).T

위에서 ``delta`` 값이 작으면 ``X`` 행렬의 두 열은 거의 선형 종속이 된다. QR 분해는 아래 조건을 만족하는 두 행렬 Q와 R을 찾는 것이다.

\begin{align}
X \beta &= y \\
QR &= X \\
Q^TQ & = QQ^T = I
\end{align}

따라서 
\begin{align}
X \beta &= y \\
QR \beta &= y \\
Q^TQR \beta & = Q^Ty = z\\
R \beta & = Q^Ty \\
R \beta & = z
\end{align}

In [ ]:
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.qr.html
Q, R = na.qr(X)
print('Q = %s' % Q)
print('R = %s' % R)
z = Q.T * y
print('z = %s' % z)

$$\beta = R^{-1} z$$

In [ ]:
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html
beta = na.solve(R, z)
print('beta = %s' % beta)

$$X \beta -y = 0$$

In [ ]:
err = X * beta - y
print('err = %s' % err)
print('||err||2 = %s' % np.sqrt(err.T * err))

한편 이론적인 접근은 다음과 같은 결과를 보여 준다.

In [ ]:
try:
    beta_left_inv = (X.T * X).I*X.T * y
except na.LinAlgError: 
    print('(X.T * X).shape = %s' % str((X.T * X).shape))
    print('rank(X.T * X) = %r' % na.matrix_rank(X.T * X))
else:
    err_left_inv = X * beta_left_inv - y
    print('err_left_inv = %s' % err_left_inv)
    print('||err_left_inv||2 = %s' % np.sqrt(err_left_inv.T*err_left_inv))